# Metagrating

The metagrating challenge entails designing a beam deflector that couples a normally-incident plane wave into one with a polar angle of 50 degrees. This problem was studied in "[Validation and characterization of algorithms and software for photonics inverse design](https://opg.optica.org/josab/ViewMedia.cfm?uri=josab-41-2-A161)" by Chen et al.; the associated [photonics-opt-testbed repo](https://github.com/NanoComp/photonics-opt-testbed) contains several example designs.

## Simulating an existing design

We'll begin by loading, visualizing, and simulating designs from the [invrs-gym paper](https://arxiv.org/abs/2410.24132).

In [ ]:
import matplotlib.pyplot as plt
import numpy as onp
from skimage import measure
from totypes import json_utils


files = [
    "240710_mfschubert_f86277871e4c38c9e6209938acee7f3c8de2cd476fd52e59a6d35e204ed81f49.json",
    "240710_mfschubert_52e918ac1c6e90bb23d24f8bb3714a582e4fe897675321a343e7fd9b48a34b07.json",
    "240710_mfschubert_a33070d2d053e3b278056e6bcd387b552e2cd619f8096ac306c3604386b27fe1.json",
]

def load_design(file):
    with open(f"../../../reference_designs/metagrating/{file}", "r") as f:
        serialized = f.read()
    params = json_utils.pytree_from_json(serialized)
    return params

designs = [load_design(file) for file in files]

plt.figure(figsize=(7, 4))
for i, design in enumerate(designs):
    ax = plt.subplot(1, 3, i + 1)
    im = ax.imshow(1 - design.array, cmap="gray")
    im.set_clim([-2, 1])
    contours = measure.find_contours(design.array)
    for c in contours:
        plt.plot(c[:, 1], c[:, 0], "k", lw=1)
    ax.set_xticks([])
    ax.set_yticks([])

Now, we'll create a `metagrating` challenge, which provides everything we need to simulate and optimize the metagrating.

In [ ]:
from invrs_gym import challenges

challenge = challenges.metagrating()

In [ ]:
# Perform simulation using component response method.
response, aux = challenge.component.response(params=designs[0])

The `response` contains the transmission and reflection efficiency into each diffraction order, and for TE- and TM-polarized cases. However, we only care about TM diffraction into the +1 order. Fortunately, the `challenge` has a `metrics` method that extracts this value.

In [ ]:
metrics = challenge.metrics(response, params=designs[0], aux=aux)
print(f"TM transmission into +1 order: {metrics['average_efficiency'] * 100:.1f}%")

Apparently, this is a design with extremely high efficiency.

Now let's take a look at the remaining designs.

In [ ]:
for i, design in enumerate(designs):
    response, aux = challenge.component.response(params=design)
    metrics = challenge.metrics(response, params=design, aux=aux)
    print(
        f"design {i + 1}: TM transmission into +1 order: {metrics['average_efficiency'] * 100:.1f}%"
    )

The efficiency is lower for the second and third designs; this is expected, as the feature within these designs are larger compared to the first design.